In [1]:
!pip install kaggle --q
from google.colab import files
files.upload()  # حمل ملف kaggle.json الخاص بحسابك من كاجل
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d dataclusterlabs/mobile-phone-image-dataset
!unzip mobile-phone-image-dataset.zip -d /content/mobile_dataset


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/dataclusterlabs/mobile-phone-image-dataset
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 80% 226M/283M [00:00<00:00, 628MB/s] 
100% 283M/283M [00:00<00:00, 609MB/s]
Archive:  mobile-phone-image-dataset.zip
  inflating: /content/mobile_dataset/Annotations/Annotations/Datacluster Labs Phone Dataset (1).xml  
  inflating: /content/mobile_dataset/Annotations/Annotations/Datacluster Labs Phone Dataset (10).xml  
  inflating: /content/mobile_dataset/Annotations/Annotations/Datacluster Labs Phone Dataset (100).xml  
  inflating: /content/mobile_dataset/Annotations/Annotations/Datacluster Labs Phone Dataset (11).xml  
  inflating: /content/mobile_dataset/Annotations/Annotations/Datacluster Labs Phone Dataset (12).xml  
  inflating: /content/mobile_dataset/Annotations/Annotations/Datacluster Labs Phone Dataset (13).xml  
  inflating: /content/mobile_dataset/Annotations/Annot

In [2]:
# Install dependencies
!pip install ultralytics --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00


In [3]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
model = YOLO('yolo11s.pt')

In [5]:
import shutil
import random
import os

main_dir = '/content/mobile_dataset'
train_dir = os.path.join(main_dir, 'train')
val_dir = os.path.join(main_dir, 'val')

# إنشاء المجلدات
for d in [train_dir, val_dir]:
    os.makedirs(os.path.join(d, 'images'), exist_ok=True)
    os.makedirs(os.path.join(d, 'labels'), exist_ok=True)


In [ ]:
yaml_content = """
path: /content/mobile_dataset
train: /content/mobile_dataset/train/images
val: /content/mobile_dataset/val/images
nc: 3 # Updated number of classes

names:
  0: mobile_phone 
"""

with open('/content/mobile_dataset/data.yaml', 'w') as f:
    f.write(yaml_content)

In [ ]:
import os
import xml.etree.ElementTree as ET

# Paths for train and validation sets
train_labels_dir = "/content/mobile_dataset/train/labels"
val_labels_dir = "/content/mobile_dataset/val/labels"

# Get class names from data.yaml - Ensure this matches your data.yaml
classes = ["mobile_phone"] 

def convert_xml_to_txt(annotations_dir, output_dir, classes):
    """Converts XML annotation files to YOLO format TXT files."""
    os.makedirs(output_dir, exist_ok=True)

    for xml_file in os.listdir(annotations_dir):
        if not xml_file.endswith(".xml"):
            continue

        tree = ET.parse(os.path.join(annotations_dir, xml_file))
        root = tree.getroot()

        image_name_element = root.find("filename")
        if image_name_element is None:
            print(f"Warning: Skipping {xml_file} as it does not contain a 'filename' element.")
            continue
        image_name = image_name_element.text
        txt_file_name = image_name.replace(".jpg", ".txt") # Assuming image names are .jpg
        txt_file_path = os.path.join(output_dir, txt_file_name)

        size = root.find("size")
        if size is None:
             print(f"Warning: Skipping {xml_file} as it does not contain a 'size' element.")
             continue
        w = int(size.find("width").text)
        h = int(size.find("height").text)

        with open(txt_file_path, "w") as f:
            for obj in root.findall("object"):
                cls_element = obj.find("name")
                if cls_element is None:
                    print(f"Warning: Skipping object in {xml_file} as it does not contain a 'name' element.")
                    continue
                cls = cls_element.text

                if cls not in classes:
                    print(f"Warning: Skipping object with unknown class '{cls}' in {xml_file}.")
                    continue

                cls_id = classes.index(cls)

                xml_box = obj.find("bndbox")
                if xml_box is None:
                    print(f"Warning: Skipping object in {xml_file} as it does not contain a 'bndbox' element.")
                    continue

                xmin_element = xml_box.find("xmin")
                ymin_element = xml_box.find("ymin")
                xmax_element = xml_box.find("xmax")
                ymax_element = xml_box.find("ymax")

                if None in [xmin_element, ymin_element, xmax_element, ymax_element]:
                     print(f"Warning: Skipping object in {xml_file} due to incomplete bounding box.")
                     continue

                # Convert bounding box coordinates to float
                xmin = float(xmin_element.text)
                ymin = float(ymin_element.text)
                xmax = float(xmax_element.text)
                ymax = float(ymax_element.text)

                x_center = (xmin + xmax) / 2 / w
                y_center = (ymin + ymax) / 2 / h
                width = (xmax - xmin) / w
                height = (ymax - ymin) / h

                f.write(f"{cls_id} {x_center} {y_center} {width} {height}\n")

# Convert annotations for both train and validation sets
print("Converting train annotations...")
convert_xml_to_txt(train_labels_dir, train_labels_dir, classes) # Convert and save in the same directory

print("\nConverting validation annotations...")
convert_xml_to_txt(val_labels_dir, val_labels_dir, classes) # Convert and save in the same directory

print("\n✅ All XML annotations converted to YOLO TXT.")

Converting train annotations...

Converting validation annotations...

✅ All XML annotations converted to YOLO TXT.


In [12]:
# Paths of raw images and labels
img_dir = '/content/mobile_dataset/Mobile_image/Mobile_image'
label_dir = '/content/mobile_dataset/Annotations/Annotations'  # Adjust based on actual label location

all_images = [f for f in os.listdir(img_dir) if f.endswith('.jpg') or f.endswith('.png')]
random.seed(42)
random.shuffle(all_images)

split_index = int(0.8 * len(all_images))
train_images = all_images[:split_index]
val_images = all_images[split_index:]

# Copy images and labels to folders
def copy_data(image_list, dest_dir):
    for img_file in image_list:
        base_name = os.path.splitext(img_file)[0]
        # Copy image
        shutil.copy(os.path.join(img_dir, img_file), os.path.join(dest_dir, 'images', img_file))
        # Copy label if exists (assuming XML format as seen in unzip output)
        label_file = base_name + '.xml'
        if os.path.exists(os.path.join(label_dir, label_file)):
            shutil.copy(os.path.join(label_dir, label_file), os.path.join(dest_dir, 'labels', label_file))
        else:
             print(f"Warning: Annotation file not found for {img_file}")


copy_data(train_images, train_dir)
copy_data(val_images, val_dir)

In [ ]:
# Train the model
model = YOLO('yolo11s.pt')
results = model.train(
    data='/content/mobile_dataset/data.yaml',
    epochs=15,
    project='yolo_train_v11',
    name='log',
    device='cpu' # Use CPU as per previous error diagnosis
)

# The trained model will be saved in the 'yolo_train_v11/log/weights/' directory
print("YOLOv11 training completed. Trained model saved in 'yolo_train_v11/log/weights/'")

Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/mobile_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=log2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

In [ ]:
import shutil
import os

# Define the path to the best weights after training
source_path = '/content/yolo_train_v11/log/weights/best.pt'

# Define the destination path to save the model
destination_path = '/content/best_mobile_yolov11_model.pt'

# Check if the source file exists and copy it
if os.path.exists(source_path):
    shutil.copy(source_path, destination_path)
    print(f"Trained model saved successfully to {destination_path}")
else:
    print(f"Error: Trained model not found at {source_path}. Please ensure training completed successfully.")